In [140]:
import geopandas as gpd
import requests
import json
import urllib.parse
import time

In [157]:
# see https://cohegis.houstontx.gov/cohgispub/rest/services/PD/EconomicDevelopment_wm/MapServer/9 for details

map_server_url = 'https://cohegis.houstontx.gov/cohgispub/rest/services/PD/EconomicDevelopment_wm/MapServer/9/query'
# where = '1=1'
fields = ['OBJECTID', 'TAX_ID', 'zipcode' ,'LANDUSE_CD' ,'USEGRP' ,'STATE_CLASS' ,'ACREAGE' ,'Shape']

object_ids = []
landuse_geojson = None

stuff_left = True
requests_sent = 0

for i in range(0, 1593000, 1000):
    start = time.time()
    requests_sent += 1
    print('request: %d' % requests_sent)
    
    query_format = '?where={}&outFields={}&returnGeometry=false&f=geojson'
    query_params = {
        'where': 'OBJECTID+BETWEEN+{}+AND+{}'.format(1, 150), 
        'outFields': ','.join(fields),
        'returnGeometry': 'false',
        'f': 'geojson'
    }
    
    where = 'OBJECTID+BETWEEN+{}+AND+{}'.format(i, i + 999)
    r = requests.get(map_server_url + query_format.format(where, ','.join(fields)))
    curr_landuse_geojson = r.json()
    
#     curr_obj_ids = [feature['properties']['OBJECTID'] for feature in curr_landuse_geojson['features']]
    
#     overlap = set(object_ids).intersection(curr_obj_ids)
#     print('overlap: %d' % len(overlap))
    
    if landuse_geojson is None:
        landuse_geojson = curr_landuse_geojson
    else:
        landuse_geojson['features'].extend(curr_landuse_geojson['features'])
    
    object_ids.extend(curr_obj_ids)
    query_params['where'] = 'OBJECTID NOT IN ({})'.format(','.join(map(str, object_ids)))
    
    print('%ss elapsed\n' % (time.time() - start))
#     stuff_left = len(curr_obj_ids) > 0

request: 1
1.4579427242279053s elapsed

request: 2
1.4649357795715332s elapsed

request: 3
1.1527299880981445s elapsed

request: 4
1.459981918334961s elapsed

request: 5
1.4843628406524658s elapsed

request: 6
1.2492048740386963s elapsed

request: 7


KeyboardInterrupt: 

In [158]:
with open('houston_lu.geojson', 'w') as outfile:
    json.dump(landuse_geojson, outfile)

In [95]:
object_ids = [feature['properties']['OBJECTID'] for feature in landuse_geojson['features']]
len(object_ids)

1000

In [91]:
query_params['where'] = 'OBJECTID NOT IN ({})'.format(','.join(map(str, object_ids)))
r = requests.post(map_server_url, data=query_params)
landuse_geojson2 = r.json()

In [155]:
# to see how test which value is the upper bound on requests
test = requests.get(map_server_url + '?where=OBJECTID%3E1592000&outFields=OBJECTID,TAX_ID,COUNTY,zipcode,LANDUSE_CD,USEGRP,STATE_CLASS,ACREAGE&returnGeometry=false&f=json').json()
len(test['features'])

956

In [159]:
gpd.read_file('houston_lu.geojson')

,OBJECTID,TAX_ID,zipcode,LANDUSE_CD,USEGRP,STATE_CLASS,ACREAGE,geometry
0,1,1004280000028,77099,1001,1,A1,0.1731,POLYGON ((-95.57538265941477 29.68389288873791...
1,2,1246850020042,77429,1001,1,A1,0.1641,POLYGON ((-95.69575228170713 29.98661410857128...
2,3,1153910020021,77450,1001,1,A1,0.1848,POLYGON ((-95.72291405670653 29.75986394661937...
3,4,1146080090003,77099,1006,2,Z3,0,POLYGON ((-95.57435084553084 29.66627394170596...
4,5,1146080200004,77099,1006,2,Z3,0,POLYGON ((-95.57435084553084 29.66627394170596...
5,6,1146080230005,77099,1006,2,Z3,0,POLYGON ((-95.57435084553084 29.66627394170596...
6,7,1246850020033,77429,1001,1,A1,0.1641,POLYGON ((-95.69731276748156 29.98652812306259...
7,8,1142490210001,77450,1001,1,A1,0.1882,POLYGON ((-95.72469805284985 29.76018353244636...
8,9,1142490220001,77450,1001,1,A1,0.1899,POLYGON ((-95.72593970660661 29.76018584678203...
9,10,1153910020001,77450,1001,1,A1,0.1791,POLYGON ((-95.72163832853741 29.76017778175756...
